# 🌸 Система прогнозирования продаж цветов

## Развертывание в Google Colab

Этот ноутбук позволяет запустить систему прогнозирования цветов прямо в Google Colab с полной интеграцией с Google Sheets.

### 📋 Что включено:
- ✅ Streamlit веб-интерфейс
- ✅ Интеграция с Google Sheets
- ✅ Система машинного обучения
- ✅ Мультимагазинная архитектура
- ✅ Ручная корректировка прогнозов

### 🚀 Инструкции по запуску:
1. Выполните все ячейки по порядку
2. Загрузите ваш файл `service_account_key.json` когда появится запрос
3. Укажите URL вашей Google Таблицы
4. Перейдите по ссылке на веб-интерфейс

## 📦 Установка зависимостей

In [ ]:
# Установка необходимых пакетов
!pip install streamlit pandas numpy plotly gspread google-auth scikit-learn requests openpyxl
!npm install -g localtunnel

## ⚙️ Настройка проекта

In [ ]:
# Клонирование проекта
!git clone https://github.com/Alanaa5800/flower-forecast.git
%cd flower-forecast

In [ ]:
# Загрузка файла ключей Google Cloud
from google.colab import files
import json

print("🔐 Загрузите ваш файл service_account_key.json:")
uploaded = files.upload()

# Проверяем, что файл загружен
if 'service_account_key.json' in uploaded:
    print("✅ Файл ключей загружен успешно!")
    
    # Проверяем структуру файла
    with open('service_account_key.json', 'r') as f:
        key_data = json.load(f)
    
    print(f"📧 Email сервисного аккаунта: {key_data['client_email']}")
    print(f"🏗️ Проект: {key_data['project_id']}")
else:
    print("❌ Файл не загружен. Пожалуйста, загрузите service_account_key.json")

In [ ]:
# Настройка URL Google Таблицы
SPREADSHEET_URL = input("📊 Введите URL вашей Google Таблицы: ")

if SPREADSHEET_URL:
    print(f"✅ URL сохранен: {SPREADSHEET_URL[:50]}...")
else:
    print("❌ URL не указан. Будет использована демо-версия")
    SPREADSHEET_URL = "DEMO_MODE"

## 🔗 Тестирование подключения

In [ ]:
# Тест подключения к Google Sheets
import gspread
from google.oauth2.service_account import Credentials
import os

def test_google_sheets_connection():
    if not os.path.exists('service_account_key.json'):
        print("❌ Файл ключей не найден")
        return False
    
    if SPREADSHEET_URL == "DEMO_MODE":
        print("📊 Режим демонстрации - Google Sheets не используется")
        return True
    
    try:
        # Подключение к Google Sheets
        scope = ['https://spreadsheets.google.com/feeds',
                'https://www.googleapis.com/auth/drive']
        
        creds = Credentials.from_service_account_file(
            'service_account_key.json', scopes=scope
        )
        
        gc = gspread.authorize(creds)
        sheet = gc.open_by_url(SPREADSHEET_URL)
        
        print(f"✅ Подключение успешно!")
        print(f"📊 Название таблицы: {sheet.title}")
        
        # Проверка листов
        worksheets = sheet.worksheets()
        print(f"📄 Найдено листов: {len(worksheets)}")
        
        for ws in worksheets:
            print(f"  - {ws.title}")
        
        return True
        
    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
        return False

connection_ok = test_google_sheets_connection()

## 📊 Демонстрация работы системы

In [ ]:
# Запуск демонстрации системы
!python3 simple_demo.py

In [ ]:
# Демонстрация обучения модели
!python3 model_training.py

## 🌐 Запуск веб-интерфейса Streamlit

In [ ]:
# Модификация Streamlit приложения для Colab
streamlit_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import json
import os

# Специальная конфигурация для Colab
st.set_page_config(
    page_title="🌸 Система прогнозирования продаж цветов (Colab)",
    page_icon="🌸",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Проверяем наличие Google Sheets интеграции
GOOGLE_SHEETS_AVAILABLE = os.path.exists('service_account_key.json')

# Загружаем основное приложение
exec(open('streamlit_forecast_app.py').read())

# Добавляем информацию о Colab
st.sidebar.markdown("---")
st.sidebar.markdown("### 🚀 Colab Deployment")
st.sidebar.info("Приложение запущено в Google Colab")

if GOOGLE_SHEETS_AVAILABLE:
    st.sidebar.success("✅ Google Sheets подключен")
else:
    st.sidebar.warning("⚠️ Демо режим (без Google Sheets)")
'''

# Сохраняем модифицированное приложение
with open('streamlit_colab_app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_code)

print("✅ Streamlit приложение подготовлено для Colab")

In [ ]:
# Запуск Streamlit в фоновом режиме
import subprocess
import time
import threading

def run_streamlit():
    subprocess.run([
        'streamlit', 'run', 'streamlit_colab_app.py',
        '--server.port', '8501',
        '--server.headless', 'true',
        '--server.enableCORS', 'false',
        '--server.enableXsrfProtection', 'false'
    ])

# Запускаем Streamlit в отдельном потоке
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.daemon = True
streamlit_thread.start()

print("🚀 Streamlit запускается...")
time.sleep(10)  # Ждем запуска
print("✅ Streamlit запущен на порту 8501")

In [ ]:
# Создание туннеля для доступа к приложению
import subprocess
import time
import re

def create_tunnel():
    try:
        # Запуск localtunnel
        result = subprocess.run(
            ['npx', 'localtunnel', '--port', '8501'],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if result.returncode == 0:
            # Извлекаем URL из вывода
            output = result.stdout
            url_match = re.search(r'https://[\w-]+\.loca\.lt', output)
            if url_match:
                return url_match.group(0)
        
        return None
        
    except Exception as e:
        print(f"Ошибка создания туннеля: {e}")
        return None

# Пытаемся создать туннель
print("🌐 Создание публичного доступа...")
tunnel_url = create_tunnel()

if tunnel_url:
    print(f"✅ Приложение доступно по адресу: {tunnel_url}")
    print("\n🎯 Инструкции:")
    print("1. Перейдите по ссылке выше")
    print("2. Нажмите 'Click to Continue'")
    print("3. Используйте систему прогнозирования!")
else:
    print("❌ Не удалось создать публичный доступ")
    print("Приложение доступно локально на порту 8501")

## 🔄 Альтернативный способ доступа

In [ ]:
# Использование ngrok как альтернативы
!pip install pyngrok

from pyngrok import ngrok
import time

try:
    # Создаем туннель через ngrok
    public_url = ngrok.connect(8501)
    print(f"🌐 Приложение доступно по адресу: {public_url}")
    print("\n🎯 Инструкции:")
    print("1. Перейдите по ссылке выше")
    print("2. Используйте систему прогнозирования!")
    print("\n⚠️ Внимание: туннель будет активен только пока работает этот ноутбук")
    
except Exception as e:
    print(f"❌ Ошибка создания ngrok туннеля: {e}")
    print("Попробуйте использовать localtunnel выше")

## 📖 Инструкция по использованию

### 🎯 Основные функции:

1. **📊 Прогноз** - просмотр прогнозов продаж
2. **✏️ Корректировки** - ручная правка прогнозов
3. **🌤️ Погода** - учет погодных условий
4. **📈 Аналитика** - анализ и метрики
5. **⚙️ Настройки** - конфигурация системы

### 🚀 Быстрый старт:

1. Перейдите по ссылке выше
2. В боковой панели выберите магазины
3. Нажмите "🔄 Обновить прогноз"
4. Изучите результаты во вкладке "📊 Прогноз"
5. При необходимости внесите корректировки во вкладке "✏️ Корректировки"

### 🔧 Интеграция с Google Sheets:

Если вы загрузили файл ключей и указали URL таблицы:
- Данные автоматически сохраняются в Google Sheets
- Можно загружать исторические данные из таблицы
- Корректировки синхронизируются с облаком

### 📞 Поддержка:

При возникновении проблем:
1. Перезапустите ячейку с Streamlit
2. Проверьте подключение к Google Sheets
3. Обратитесь к документации в репозитории

## 📊 Мониторинг системы

In [ ]:
# Мониторинг состояния системы
import requests
import time
import psutil

def check_system_status():
    print("🔍 Проверка состояния системы...")
    print("=" * 40)
    
    # Проверка Streamlit
    try:
        response = requests.get('http://localhost:8501', timeout=5)
        if response.status_code == 200:
            print("✅ Streamlit: Работает")
        else:
            print(f"⚠️ Streamlit: Код ответа {response.status_code}")
    except:
        print("❌ Streamlit: Не отвечает")
    
    # Проверка файлов
    files_to_check = [
        'streamlit_forecast_app.py',
        'google_sheets_integration.py',
        'multi_store_architecture.py',
        'model_training.py'
    ]
    
    for file in files_to_check:
        if os.path.exists(file):
            print(f"✅ {file}: Найден")
        else:
            print(f"❌ {file}: Отсутствует")
    
    # Проверка Google Sheets
    if os.path.exists('service_account_key.json'):
        print("✅ Google Sheets: Ключи найдены")
    else:
        print("⚠️ Google Sheets: Ключи не загружены")
    
    # Использование ресурсов
    memory = psutil.virtual_memory()
    print(f"💾 Память: {memory.percent}% ({memory.used // 1024**2} MB)")
    
    print("\n🕐 Время проверки:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

check_system_status()

In [ ]:
# Поддержание работы системы
print("🔄 Система работает...")
print("💡 Оставьте эту ячейку запущенной для поддержания работы приложения")
print("⏹️ Для остановки прервите выполнение ячейки")

try:
    while True:
        time.sleep(300)  # Проверка каждые 5 минут
        print(f"⏰ {datetime.now().strftime('%H:%M:%S')} - Система активна")
        
        # Проверяем, что Streamlit все еще работает
        try:
            response = requests.get('http://localhost:8501/_stcore/health', timeout=5)
            if response.status_code != 200:
                print("⚠️ Streamlit возможно завис, перезапуск...")
                # Здесь можно добавить логику перезапуска
        except:
            print("❌ Streamlit не отвечает")
            
except KeyboardInterrupt:
    print("\n🛑 Система остановлена пользователем")
except Exception as e:
    print(f"\n❌ Ошибка системы: {e}")